In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sqlalchemy import create_engine

import tensorflow as tf
import keras
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

In [18]:
engine = create_engine("sqlite:///../data/price_database.db")

In [19]:
raw_data = pd.read_sql("SELECT * FROM BTC_price_1m", engine)
raw_data.set_index("Datetime", inplace=True)

In [20]:
x = raw_data.drop(["Close", "Adj Close", "pct_change"], axis=1)
y = raw_data.filter(["pct_change"], axis=1)

In [21]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
y_scaled = scaler.fit_transform(y)

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.1)

In [23]:
def create_sequences(X_data, Y_data, window_size):
    X, Y = [], []
    for i in range(len(X_data) - window_size):
        X.append(X_data[i:i + window_size])
        Y.append(Y_data[i + window_size])
    return np.array(X), np.array(Y)

In [24]:
x_train, y_train = create_sequences(x_train, y_train, 15)
x_test, y_test = create_sequences(x_test, y_test, 15)

In [25]:
x_test.shape, y_test.shape

((919, 15, 4), (919, 1))

In [38]:
def model_builder(hp):
    model = Sequential()

    # input layer
    model.add(LSTM(units=x_train.shape[1], input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))

    # let the model decide how many LSTM layers it wants to have
    for i in range(hp.Int("num_LSTM_layers", min_value=1, max_value=2, step=1)):
        model.add(
            Dropout(hp.Float("dropout1_" + str(i), min_value=0, max_value=0.5, step=0.1))
        )
        model.add(
            LSTM(
                units=hp.Int("lstm_layer_" + str(i), min_value=32, max_value=128, step=16),
                activation=hp.Choice("act_" + str(i), ["relu", "sigmoid"]),
                return_sequences=True
            )
        )
        
    model.add(LSTM(units=10, return_sequences=False))
      
    # let the model decide how many LSTM layers it wants to have  
    for i in range(hp.Int("num_dense_layers", min_value=1, max_value=2, step=1)):
        model.add(
            Dropout(hp.Float("dropout2_" + str(i), min_value=0, max_value=0.5, step=0.1))
        )
        model.add(
            Dense(
                units=hp.Int("dense_layer_" + str(i), min_value=64, max_value=1024, step=64),
                activation=hp.Choice("act_" + str(i), ["relu", "sigmoid"]),
            )
        )

    # output shape of the model the same as the number of features
    model.add(
        Dense((1), activation="relu"),
    )

    # compile the model
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-1, 1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss="mse",
    )

    return model


In [39]:
earlyStop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)


In [40]:
tuner = kt.RandomSearch(
    model_builder,
    objective="val_loss",
    max_trials=20,
    directory=r"../logs/",
    project_name="RandomSearch v1.08",
)


In [41]:
tuner.search(
    x_train,
    y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.2,
    callbacks=[earlyStop],
)


Trial 2 Complete [00h 00m 23s]
val_loss: 1.1116660833358765

Best val_loss So Far: 1.1116660833358765
Total elapsed time: 00h 00m 51s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_LSTM_layers
0.1               |0                 |dropout1_0
48                |112               |lstm_layer_0
sigmoid           |sigmoid           |act_0
2                 |2                 |num_dense_layers
0.2               |0.1               |dropout2_0
768               |576               |dense_layer_0
0.001             |0.1               |learning_rate
0.4               |0                 |dropout1_1
80                |32                |lstm_layer_1
relu              |relu              |act_1
0.3               |0                 |dropout2_1
512               |64                |dense_layer_1

Epoch 1/10
105/105 [==============================] - 13s 71ms/step - loss: 1.0437 - val_loss: 1.1117
Epoch 2/10
105/105 [============

KeyboardInterrupt: 

In [ ]:
hpModel = tuner.get_best_models()[0]
hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

hpModel.summary(), hyperparameters.values

In [ ]:
trained_model = hpModel.fit(
    x_scaled, y_scaled, epochs=2, batch_size=32, callbacks=earlyStop, validation_split=0.2)

In [ ]:
fig, (ax1) = plt.subplots(figsize=(6, 4))
fig.suptitle("Evaluation of Model Training")
ax2 = ax1.twinx()

ax1.plot(trained_model.history["val_loss"], label="Val_loss")
ax1.set_ylabel("Val_loss")
ax1.set_xlabel("Epoch")

ax2.plot(trained_model.history["loss"], color="r", label="Loss")
ax2.set_ylabel("Loss")
ax2.set_xlabel("Epoch")

fig.legend()

In [ ]:
y_pred = hpModel.predict(x_test)

In [ ]:
y_pred = pd.DataFrame(scaler.inverse_transform(y_pred))

In [ ]:
results = pd.DataFrame()
results["y_true"] = y_test
results["y_pred"] = y_pred.values

In [ ]:
mean_squared_error(results["y_true"], results["y_pred"])

In [ ]:
results.plot()